## Data Extraction and load from FRED API.. 

In [372]:
## Import packages for the process... 

import requests
import pickle
import os
import mysql.connector
import time

### Using pickle to wrap the database credentials and Fred API keys 

In [35]:
if not os.path.exists('fred_api_secret.pk1'):
    fred_key = {}
    fred_key['api_key'] = ''
    with open ('fred_api_secret.pk1','wb') as f:
        pickle.dump(fred_key,f)
else:
    fred_key=pickle.load(open('fred_api_secret.pk1','rb'))

In [39]:
if not os.path.exists('fred_sql.pk1'):
    fred_sql = {}
    fred_sql['user'] = ''
    fred_sql['password'] = ''
    fred_sql['database'] = ''
    with open ('fred_sql.pk1','wb') as f:
        pickle.dump(fred_sql,f)
else:
    fred_sql=pickle.load(open('fred_sql.pk1','rb'))

#### testing database connection.

We have a lookup table containing the FRED series along with the value. Let's export the connection parameters and test the connection by running a select query against the lookup table. 


In [374]:
cn = mysql.connector.connect(user=fred_sql['user'], password=fred_sql['password'],
                              host='127.0.0.1',
                              database=fred_sql['database'])
cursor = cn.cursor()

query = ("SELECT frd_cd,frd_val FROM frd_lkp")
cursor.execute(query)

for (frd_cd,frd_val) in cursor:
    sr_list.append(frd_cd)
    print(frd_cd +' - '+ frd_val)

cn.close()

UMCSENT - University of Michigan Consumer Sentiment Index
GDPC1 - Real Gross Domestic Product
UNRATE - US Civilian Unemployment Rate


## Helper functions.. 

We are doing this exercise with minimal modelling. Hence, just one target table to store the observations for all series. 

Let's create few helper functions to make this process easier. 

    db_max_count - We are adding surrogate key to the table to make general querying operations and loads easier. COALESCE is used, to get a valid value from the database. 

    db_srs_count - Since we are using just one target table, we are adding the series name as part of the data. this function will help us with the count for each series present in the table. 

    fred_req - Helper function that sends the request to FRED API and returns the response back.. 


In [375]:
def db_max_count():
    cn = mysql.connector.connect(user=fred_sql['user'], password=fred_sql['password'],
                              host='127.0.0.1',
                              database=fred_sql['database'])
    cursor = cn.cursor()
    dbquery = ("SELECT COALESCE(max(idfrd_srs),0) FROM frd_srs_data")
    cursor.execute(dbquery)
    
    for ct in cursor:
        if ct is not None:
            return ct[0]
    cn.close()

In [376]:
def db_srs_count():
    cn = mysql.connector.connect(user=fred_sql['user'], password=fred_sql['password'],
                              host='127.0.0.1',
                              database=fred_sql['database'])
    cursor = cn.cursor()
    dbquery = ("SELECT frd_srs, count(*) FROM frd_srs_data group by frd_srs")
    cursor.execute(dbquery)
    
    for ct in cursor:
        print(ct)
    cn.close()

In [398]:
def fred_req(series):
    time.sleep(10)
    response = requests.get('https://api.stlouisfed.org/fred/series/observations?series_id='+series+'&api_key='+fred_key['api_key']+'&file_type=json')
    result = response.json()
    return result

## Main functions.. 

We are creating main functions to support the process. Here are the steps 

    1) Get the data from FRED API. (helper function created above)
    2) Validate and transform the observations data from API.
    3) Create tuples according to the table structure. 
    4) Load the tuples into the relational database
    
fred_data for Step 2 & Step 3. Function dbload for Step 4. 

In [384]:
def dbload(tuple_list):
    try:
    
        cn = mysql.connector.connect(user=fred_sql['user'], password=fred_sql['password'],
                                      host='127.0.0.1',
                                      database=fred_sql['database'])
        cursor = cn.cursor()

        insert_query = ("INSERT INTO frd_srs_data"
                               "(idfrd_srs,frd_srs,frd_srs_val_dt,frd_srs_val,frd_srs_val_yr,frd_srs_val_mth,frd_srs_val_dy,frd_srs_strt_dt,frd_srs_end_dt)"
                               "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)")

        print("*** Database Connection Initialized, buckle up the seat belts..")
    
        # Data load.. 
        for i in range(len(tuple_list)):
            data_val=tuple_list[i]
            cursor.execute(insert_query, data_val)

        cn.commit()
        
        ## Intended timeout before starting the next interation of load..  
        time.sleep(5)
        
        
        print("\n *** Data load successful.. ")
        db_srs_count()
        
        # Closing database connection... 
        cn.close
    except mysql.connector.Error as err:
        cn.close
        print("Something went wrong: {}".format(err))

In [443]:
def fred_data(series):
    print("\n")
    print("** Getting data for the series: " + series)
    
    counter=db_max_count()
    # Calling function to get the data from FRED API for the series.
    fred_result = fred_req(series)
    
    print("** Number of observations extracted -" '{:d}'.format(fred_result['count']))
    
    # transforming observations and preparing for data load.
    print("** Preparing data for load for series -",series)
    temp_lst = fred_result['observations']
    tlist = []

    # from the incoming data, let's create tuple of values for data load. 
    for val in range(len(temp_lst)):
        temp_dict = temp_lst[val]
        for key,val in temp_dict.items():
            if key=='date':
                dt_lst = val.split("-")
                yr  = dt_lst[0]
                mth = dt_lst[1]
                dtt = dt_lst[2]
            if key=='value':
                if len(val.strip())>1:
                    out_val = val
                else:
                    out_val = 0.00
                counter+=1
        tup = (counter,series,temp_dict['date'],out_val,yr,mth,dtt,temp_dict['realtime_start'],temp_dict['realtime_end'])
        tlist.append(tup)
    print("** Data is ready for the load.. Loading " '{:d}'.format(len(tlist)))
    dbload(tlist)

### Starting point... 

So, we have all functions created based on few assumptions (that data is all good with very minimal or no issues). 

In [444]:
sr_list = ['UMCSENT', 'GDPC1', 'UNRATE']

for series in sr_list:
    fred_data(series)



** Getting data for the series: UMCSENT
** Number of observations extracted -574
** Preparing data for load for series - UMCSENT
** Data is ready for the load.. Loading 574
*** Database Connection Initialized, buckle up the seat belts..

 *** Data load successful.. 
('UMCSENT', 574)


** Getting data for the series: GDPC1
** Number of observations extracted -284
** Preparing data for load for series - GDPC1
** Data is ready for the load.. Loading 284
*** Database Connection Initialized, buckle up the seat belts..

 *** Data load successful.. 
('GDPC1', 284)
('UMCSENT', 574)


** Getting data for the series: UNRATE
** Number of observations extracted -842
** Preparing data for load for series - UNRATE
** Data is ready for the load.. Loading 842
*** Database Connection Initialized, buckle up the seat belts..

 *** Data load successful.. 
('GDPC1', 284)
('UMCSENT', 574)
('UNRATE', 842)


In [445]:
    cn = mysql.connector.connect(user=fred_sql['user'], password=fred_sql['password'],
                              host='127.0.0.1',
                              database=fred_sql['database'])
    cursor = cn.cursor()
    quizquery = ("SELECT frd_srs_val_yr , avg(frd_srs_val) as avg_unrate FROM fred.frd_srs_data WHERE frd_srs='UNRATE' AND frd_srs_val_yr BETWEEN 1980 AND 2015 GROUP BY  frd_srs_val_yr  ORDER BY 1")
    cursor.execute(quizquery)
    
    for qz in cursor:
        print(qz)

(1980, 7.175000000000001)
(1981, 7.616666666666667)
(1982, 9.708333333333332)
(1983, 9.6)
(1984, 7.508333333333334)
(1985, 7.191666666666666)
(1986, 7.0)
(1987, 6.175000000000001)
(1988, 5.491666666666666)
(1989, 5.258333333333333)
(1990, 5.616666666666666)
(1991, 6.849999999999999)
(1992, 7.491666666666667)
(1993, 6.908333333333332)
(1994, 6.1000000000000005)
(1995, 5.591666666666668)
(1996, 5.408333333333334)
(1997, 4.941666666666666)
(1998, 4.5)
(1999, 4.216666666666668)
(2000, 3.9666666666666663)
(2001, 4.741666666666666)
(2002, 5.783333333333334)
(2003, 5.991666666666667)
(2004, 5.541666666666667)
(2005, 5.083333333333333)
(2006, 4.608333333333333)
(2007, 4.616666666666667)
(2008, 5.8)
(2009, 9.283333333333333)
(2010, 9.608333333333333)
(2011, 8.933333333333334)
(2012, 8.075000000000001)
(2013, 7.358333333333334)
(2014, 6.175000000000001)
(2015, 5.266666666666667)
